# Copy Memory

In this task, each input sequence has length T + 20. The first 10 values are chosen randomly among the digits 1, . . . , 8, with the rest being all zeros, except for the last 11 entries that are filled with the digit ‘9’ (the first ‘9’ is a delimiter). The goal is to generate an output of the same length that is zero everywhere except the last 10 values after the delimiter, where the model is expected to repeat the 10 values it encountered at the start of the input.

Ex.

Input  = 1234567876 000000000 99999999999 
         
         Message        T     Delimeter+Space

Output = 0000000000 000000000 01234567876

                              Message


In [ ]:
def Creas